# Introduction

# Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models, utils

import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
# Need to get Google Drive access
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Load the dataset into a Pandas dataframe
img_dir = os.path.join('/content/gdrive/My Drive/data/breast_cancer_nuclei/patches_64')

In [ ]:
args = {}

# Training and testing batch size
args["train_batch_size"] = 8 # 64
args["test_batch_size"] = 8 # 1000

# How long to train for
args["epochs"] = 2 # 100

# Learning rate: "Speed" with which the optimizer adjusts weights
args["lr"] = 0.01

# Momentum: How quickly the weights respond to changing gradients
args["momentum"] = 0.5

# Whether to use CUDA or not
args["no_cuda"] = False

# Seed for reproducible training
args["seed"] = 1

# How often to spit out log / progress updates
args["log_interval"] = 10

# Whether to save the trained model
args["save_model"] = False

# Decide whether to use CUDA
use_cuda = not args["no_cuda"] and torch.cuda.is_available()

# Set the seed
torch.manual_seed(args["seed"])

# Select the device to use based on the `use_cuda` flag
device = torch.device("cuda" if use_cuda else "cpu")

# Keyword arguments for the dataloader
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [ ]:
data_transform = transforms.Compose(
    [transforms.Resize(64),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

nuclei_trainset = datasets.ImageFolder(root=os.path.join(img_dir, 'train'), transform=data_transform)
nuclei_testset = datasets.ImageFolder(root=os.path.join(img_dir, 'test'), transform=data_transform)

nuclei_trainloader = torch.utils.data.DataLoader(nuclei_trainset, batch_size=args['train_batch_size'],
                                                 shuffle=True, num_workers=2)
nuclei_testloader = torch.utils.data.DataLoader(nuclei_trainset, batch_size=args['test_batch_size'],
                                                 shuffle=False, num_workers=2)

classes = ('nonnuclei', 'nuclei')

## Visualize Some Images

In [ ]:
def imshow(images):
    img_grid = utils.make_grid(images)
    img = img_grid / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    
    fig, ax = plt.subplots(figsize=(20,10))
    ax.imshow(np.transpose(npimg, (1, 2, 0)))
    ax.axis('off')
    plt.show()

# Data Description and Access

# Data Visualization and Exploration

In [ ]:
# Get some random training images (one iteration of the dataloader)
dataiter = iter(nuclei_trainloader)
images, labels = dataiter.next()
imshow(images)

# Print the associated labels
print('\t' + '\t\t'.join('%5s' % classes[labels[j]] for j in range(args['train_batch_size'])))
print(' ')
print('The size of the image batch is: {}'.format(images.shape))
print('This represents (batch_size, channels, height, width)')

## Model Definition

In [ ]:
class NucleiNet(nn.Module):
    def __init__(self, disp_size):
        super(NucleiNet, self).__init__()
        
        # Flag whether or not to print out information about the tensor
        self.disp_size = disp_size
        
        # nn.Conv2d(in_channels, out_channels, kernel_size)
        self.conv1 = nn.Conv2d(3, 6, 3, 1, 1)
        
        # nn.MaxPool2d(kernel_size, stride)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3, 1, 1)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        # nn.Linear(in_features, out_features)
        self.fc1 = nn.Linear(16 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 120)
        self.fc3 = nn.Linear(120, 84)
        self.fc4 = nn.Linear(84, 2)

    def forward(self, x):
        if self.disp_size:
            print('x input size:\t\t\t\t\t{}'.format(x.shape))

        x = self.pool1(F.relu(self.conv1(x)))
        if self.disp_size:
            print('After first block [Conv->Relu->Pool]:\t\t{}'.format(x.shape))
        
        x = self.pool2(F.relu(self.conv2(x)))
        if self.disp_size:
            print('After second block [Conv->Relu->Pool]:\t\t{}'.format(x.shape))

        x = x.view(-1, 16 * 16 * 16)
        if self.disp_size:
            print('After reshape:\t\t\t\t\t{}'.format(x.shape))

        x = F.relu(self.fc1(x))
        if self.disp_size:
            print('After first linear layer:\t\t\t{}'.format(x.shape))

        x = F.relu(self.fc2(x))
        if self.disp_size:
            print('After second linear layer:\t\t\t{}'.format(x.shape))
            
        x = F.relu(self.fc3(x))
        if self.disp_size:
            print('After third linear layer:\t\t\t{}'.format(x.shape))
            
        x = self.fc4(x)
        if self.disp_size:
            print('After fourth linear layer:\t\t\t{}'.format(x.shape))
            print(' ')
        return x

## Model Interrogation

In [ ]:
# Create a model and set the "disp_size" to True, so it will print out the size of each layer
nuclei_net = NucleiNet(disp_size=True)

# Run an image batch through just to get some output
_ = nuclei_net(images)

In [ ]:
# In PyTorch you can list out the different layers as "children" of the model
list(nuclei_net.children())[0:4]

In [ ]:
# You can also pull out specific layers of the model and use them to build a new one
# Here we look at the first four layers, which include the two convolutional and pooling layers
nuclei_features = nn.Sequential(*list(nuclei_net.children())[0:4])

print("First three layers:")
print(nuclei_features)

## Visualizing Filter Blocks

In [ ]:
outputs = nuclei_features(images)
print("size of outputs: {}".format(outputs.shape))

In [ ]:
# Which image in the batch do you want to look at?
target_img = 0

# Set up the filter block
num_channels = outputs.shape[0]

# Set up the display of the filter block for this image
rows = int(np.floor(np.sqrt(num_channels)))
if np.mod(np.sqrt(num_channels), 1) != 0:
    # There is a remainder
    cols = rows + 1
else:
    cols = rows

# Plot the original
fig, ax = plt.subplots(figsize=(5,5))
plt.imshow(np.transpose(images[target_img].cpu() / 2 + 0.5, (1,2,0)))
plt.title('Original image')
plt.axis('off')
plt.tight_layout()

output_numpy = outputs[target_img,:,:,:].detach().cpu()

fig, ax = plt.subplots(rows,cols, figsize=(10,10))

for i, r in enumerate(ax):
    for j, c in enumerate(r):
        c.imshow(output_numpy[i*cols+j,:,:], cmap=plt.cm.gray)
        c.set_title('Filter {}'.format(i*cols+j))
        c.axis('off')
        
plt.tight_layout()
plt.show()

# Training

In [ ]:
nuclei_net = NucleiNet(disp_size=False)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Device: {}".format(device))

# move model to the right device
nuclei_net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(nuclei_net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
list_loss = []
avg_loss = []
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(nuclei_trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        #inputs, labels = data

        # Move to the GPU
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = nuclei_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 20 mini-batches
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 20))
            list_loss.append(running_loss / 20)
            running_loss = 0.0
    
    # Record average loss for this epoch
    avg_loss.append(np.mean(list_loss))

print('Finished Training')

In [ ]:
plt.plot(avg_loss)

In [ ]:
#dataiter = iter(nuclei_testloader)
images, labels = dataiter.next()
images, labels = images.to(device), labels.to(device)

outputs = nuclei_net(images)
_, predicted = torch.max(outputs, 1)

# print images
imshow(images.cpu())
print('GroundTruth: ', ' '.join('\t%5s' % classes[labels[j]] for j in range(args['test_batch_size'])))
print('Predicted: ', ' '.join('\t%5s' % classes[predicted[j]]
                              for j in range(args['test_batch_size'])))


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in nuclei_testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = nuclei_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the {} testing images: {} %'.format(
    total, 100 * correct / total))

In [ ]:
class_correct = list(0. for i in range(3))
class_total = list(0. for i in range(3))
with torch.no_grad():
    for data in nuclei_testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = nuclei_net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(2):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(len(classes)):
    print('Accuracy of {} : {} %'.format(
        classes[i], 
        100.0 * class_correct[i] / class_total[i]))